In [ ]:
import pandas as pd
import csv
import re

import nltk
nltk.download('punkt')
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
!pip install pymorphy2
!pip install pymystem3
# !pip install git+https://github.com/nlpub/pymystem3
# !pip install -q http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
from pymystem3 import Mystem
import pymorphy2
from string import punctuation

!pip install pandarallel
from pandarallel import pandarallel

from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/My\ Drive/SNA2020/Project_kremlinbots

df_bots=pd.read_csv('./bots_update.tsv',sep="\t",header=0)
df_bots.head()

df_politics = pd.read_csv('./politics_update.tsv',sep="\t",header=0)
df_bots.head()

emoticons = [u'&#_128514;', u'&#_128517;', u'u&#34;\U0001F923&#34;']

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F923"
        u"\U0001F9A0"
        u"\U0001F000-\U0001FFFF"
        u"\U00002639"
                           "]+", flags=re.UNICODE)

def preprocess_text_mention_and_n(text):
  if text.find(" ") - 1 > 0 and text[text.find(" ") - 1] == ',':
    text = text[text.find(" ") + 1:]
  text = text.replace('\\n', ' ')
  return text

def clean_emoticons(text):
  for word in emoticons:
    text = text.replace(word, '')
  text = emoji_pattern.sub(r'', text)
  return text

def preprocess_text_lemmatize(text, setting="pymorphy"):
    text = text.lower()
    if text == '@@@':
      return text
    if setting == "mystem":
      mystem = Mystem()
      tokens = mystem.lemmatize(text)
    elif setting == "pymorphy":
      tokens = word_tokenize(text, language="russian")
      morph = pymorphy2.MorphAnalyzer()
      tokens = [morph.parse(token)[0].normal_form for token in tokens]
    else:
       raise Exception('parameter setting should be fill')
    tokens = [token for token in tokens if token.strip() not in punctuation]
    tokens = " ".join(tokens)
    tokens = tokens.replace('-', ' ').replace("``", '').replace("''", '').replace(".", '').replace("«", '').replace("»", '').replace("—", '').replace("№", '')
    for symbol in punctuation:
      tokens = tokens.replace(symbol, '')
    for symbol in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']:
      tokens = tokens.replace(symbol, '')
    return ' '.join(tokens.split())


pandarallel.initialize()

df_bots['pymorphy']=df_bots['text'].parallel_apply(preprocess_text_mention_and_n)
df_bots['pymorphy']=df_bots['pymorphy'].parallel_apply(clean_emoticons)
df_bots['pymorphy']=df_bots['pymorphy'].parallel_apply(preprocess_text_lemmatize)
df_bots.to_csv('bots_pymorphy.tsv',sep='\t', quoting=csv.QUOTE_NONE, index=False)

df_politics['pymorphy']=df_politics['text'].parallel_apply(preprocess_text_mention_and_n)
df_politics['pymorphy']=df_politics['pymorphy'].parallel_apply(clean_emoticons)
df_politics['pymorphy']=df_politics['pymorphy'].parallel_apply(preprocess_text_lemmatize)
df_politics.to_csv('politics_pymorphy.tsv',sep='\t', quoting=csv.QUOTE_NONE, index=False)

